In [5]:
!pip install --upgrade numpy scipy GPy GPyOpt

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires n

In [15]:
import yfinance as yf
import pandas as pd
import numpy as np

# Fetch S&P 500 ETF (SPY), Gold ETF (GLD)
tickers = ["SPY", "QQQ", "GLD"]
start_date = "2013-01-01"
end_date = "2023-12-31"

data = yf.download(tickers, start=start_date, end=end_date)

prices = data["Close"]

# print(data.head())
print(prices.head())

[*********************100%***********************]  3 of 3 completed


Ticker             GLD        QQQ         SPY
Date                                         
2013-01-02  163.169998  60.452332  117.624809
2013-01-03  161.199997  60.137486  117.359070
2013-01-04  160.440002  59.939575  117.874496
2013-01-07  159.429993  59.957558  117.552361
2013-01-08  160.559998  59.840599  117.214119


In [16]:
returns = prices.asfreq('D').interpolate(method='time')

In [6]:
import numpy as np
import GPyOpt
import GPy
import GPyOpt.models
import GPyOpt.core
from nopast import get_best_evaluation

returns = returns[['GLD', 'QQQ', 'SPY']].values

AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

In [ ]:
def portfolio_sharpe(weights):
    if len(weights.shape) == 1:
        weights = weights.reshape(1, -1)
    sharpe_values = []
    for w in weights:
        w1, w2 = w[0], w[1]
        w3 = 1 - w1 - w2
        if w1 < 0 or w2 < 0 or w3 < 0:
            sharpe = 1e10  # Penalize invalid weights
        else:
            # Calculate portfolio returns
            port_ret = returns[:, 0]*w1 + returns[:, 1]*w2 + returns[:, 2]*w3
            mean_ret = port_ret.mean()
            std_ret = port_ret.std()
            sharpe = (mean_ret / std_ret) if std_ret != 0 else 0
            sharpe = -sharpe  # Convert to minimization
        sharpe_values.append(sharpe)
    return np.array(sharpe_values).reshape(-1, 1)

In [ ]:
space = GPyOpt.Design_space([
    {'name': 'w1', 'type': 'continuous', 'domain': (0, 1)},
    {'name': 'w2', 'type': 'continuous', 'domain': (0, 1)},
], constraints=[{'name': 'sum_constraint', 'constraint': 'x[:,0] + x[:,1] <= 1.0'}])

In [ ]:
X_init = np.array([
    [0.333, 0.333],  # Equal weights
    [0.0, 0.0],      # All in SPY
    [0.0, 1.0],      # All in QQQ (w3 = 0)
    [1.0, 0.0],      # All in GLD (w3 = 0)
])
y_init = portfolio_sharpe(X_init)

In [ ]:
acquisitions = [
    {
        'type': 'ei',
        'epsilon': 1e-2
    },
    {
        'type': 'pi',
        'epsilon': 1e-2
    },
    {
        'type': 'lcb',
        'upsilon': 0.2,
        'delta': 0.1
    },
]

In [ ]:
evaluations, scores = get_best_evaluation(X_init, y_init, space, acquisitions, portfolio_sharpe, factor=0.7, iterations=100, eta=4)